# IBM October 2025 Challenge

https://research.ibm.com/haifa/ponderthis/challenges/October2025.html

## 0. Introduction

A much shorter version of the problem statement could be:  
**How many spanning trees are there in an $n \times m$ maze where each node is connected to its up/down/left/right neighbors (if they exist)?**

The solution is based on **Kirchhoff's theorem** (see [Wikipedia](https://en.wikipedia.org/wiki/Kirchhoff%27s_theorem)).  
The theorem concerns the number of spanning trees in a graph, stating that this number can be computed in polynomial time from the determinant of a submatrix of the graph's Laplacian matrix — specifically, **the number equals any cofactor of the Laplacian matrix**.

We can, of course, apply Kirchhoff's theorem to our specific graph: an $n \times m$ grid where each node is connected to its up/down/left/right neighbors (if they exist). In our approach, we compute that number by evaluating a cofactor of its Laplacian matrix directly, rather than using the eigenvalue-based method.


## 1. The Solution

The solution is implemented in the method `number_spanning_trees(n, m)`.

We first compute the Laplacian matrix using sparse matrices in `laplacian_grid_sparse`.

Let's call this matrix `Lap`. Since we want to compute a (any) cofactor, we take for instance `Lap[1:,1:]`, removing the first row and column. So, we want to compute
$$
x = \text{Lap}[1:,1:].
$$

We perform the LU decomposition so that we have $ \text{Lap}[1:,1:] = LU $, where $L$ is a lower triangular matrix and $U$ is an upper triangular matrix. Thus, we have $x = \det L \cdot \det U$, and since both $L$ and $U$ are triangular, we have `det L = L.diagonal().prod()` and `det U = U.diagonal().prod()`. Therefore,
$$
x = L.\text{diagonal()}. \text{prod()} \times U.\text{diagonal()}. \text{prod()}.
$$

However, computing this directly would cause overflow. Since we want to express the solution in scientific notation $(a.bcd \times 10^N)$,  
we take the logarithm base 10 on both sides, obtaining
$$
\log_{10} x = \text{np.log10(diagL).sum()} + \text{np.log10(diagU).sum()}.
$$

Now, instead of computing $10^{\text{this number}}$ (which would overflow), we extract the mantissa $m \in [1,10)$ and the exponent $p \in \mathbb{N}$ such that
$$
x = m \times 10^p,
$$
as desired. This is computed using  `frac, expo = np.modf(logdet10)`, `mantissa = 10**frac`, and `expo = int(expo)`.

In [1]:
import numpy as np
import math
from scipy.sparse.linalg import splu
from scipy.sparse import lil_matrix, csc_matrix

In [2]:
def laplacian_grid_sparse(n, m):
    """
    Construct the Laplacian matrix of an n x m grid graph in sparse CSC format.
    Each node is connected to its up/down/left/right neighbors (if they exist).
    Diagonal entries are the degree of each node, off-diagonals are -1 for edges.
    Using LIL format for assignment, then converted to CSC for efficient LU decomposition.
    """
    N = n * m
    L = lil_matrix((N, N), dtype=int)

    def idx(i, j):
        return i * m + j    #map (i, j) -> single index

    for i in range(n):
        for j in range(m):
            k = idx(i, j)
            neighbors = []
            if i > 0: neighbors.append(idx(i-1, j))
            if i < n-1: neighbors.append(idx(i+1, j))
            if j > 0: neighbors.append(idx(i, j-1))
            if j < m-1: neighbors.append(idx(i, j+1))

            for nb in neighbors:
                L[k, nb] = -1
            L[k, k] = len(neighbors)

    return L.tocsc()  # convert to CSC directly for splu


In [3]:
def number_spanning_trees(n,m):
    """
    Returns the number of spanning trees in an n x m grid graph.
    Each node is connected to its up/down/left/right neighbors (if they exist).
    """
    Lap = laplacian_grid_sparse(n,m)
    # LU decomposition,
    lu = splu(Lap[1:,1:])# ,permc_spec='MMD_AT_PLUS_A')#

    diagL = lu.L.diagonal()
    diagU = lu.U.diagonal()

    # log10 of determinant
    logdet10 = np.log10(diagL).sum() + np.log10(diagU).sum()

    # split into mantissa and exponent
    frac, expo = np.modf(logdet10)
    mantissa = 10**frac
    expo = int(expo)

    print(f"For a size of {n}x{m}, the number of mazes is {mantissa:.3f}e{expo}.") #Equiv: {mantissa:.10f} × 10^{expo}.")

In [4]:
pairs = [(1,1),(2,2),(4,3),(3,3), (10,15),(42,57),(342,357)]
import time
for n,m in pairs:
    t0 = time.time()
    number_spanning_trees(n,m)
    print(f"elapsed time for a {n}x{m} maze is: {time.time()-t0:.2f} s")
    print("...")


For a size of 1x1, the number of mazes is 1.000e0.
elapsed time for a 1x1 maze is: 0.02 s
...
For a size of 2x2, the number of mazes is 4.000e0.
elapsed time for a 2x2 maze is: 0.00 s
...
For a size of 4x3, the number of mazes is 2.415e3.
elapsed time for a 4x3 maze is: 0.00 s
...
For a size of 3x3, the number of mazes is 1.920e2.
elapsed time for a 3x3 maze is: 0.00 s
...
For a size of 10x15, the number of mazes is 1.289e66.
elapsed time for a 10x15 maze is: 0.00 s
...
For a size of 42x57, the number of mazes is 1.148e1174.
elapsed time for a 42x57 maze is: 0.04 s
...
For a size of 342x357, the number of mazes is 1.609e61571.
elapsed time for a 342x357 maze is: 3.99 s
...
